<h1><center> Модерация токсичных комментариев.

### Описание данных:
1. text - текст комментария;
2. toxic — является ли комментарий негативным.

# 1. Подготовка

Установим и загрузим все необходимые библиотеки:

In [1]:
!pip install spacy -q 
!python3 -m spacy download en_core_web_sm

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 12.0 MB 61 kB/s eta 0:00:0101
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.0-py3-none-any.whl size=12048607 sha256=a2fe65faf433db17acd8a1f1d91a8ff218f7c26b7edaa924c74a7f59a8564b68
  Stored in directory: /tmp/pip-ephem-wheel-cache-ft0ekzrz/wheels/71/4a/56/e48f8ad9359a6780edd8cdd42955519b1a21d6365ad15628a2
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import numpy as np
from numpy.random import RandomState

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm

import warnings
warnings.filterwarnings('ignore')

import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import lightgbm as lgbm
from lightgbm import LGBMClassifier

In [3]:
pd.set_option('display.max_columns', None) 

Прочитаем файл /datasets/toxic_comments.csv c исходными данными и сохраним его в переменной comments_data. Получим первые 5 строк таблицы:

In [4]:
comments_data = pd.read_csv('/datasets/toxic_comments.csv')
comments_data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Получим общую информацию о данных:

In [5]:
comments_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Приведем текст к нижнему регистру:

In [6]:
comments_data['text'] = comments_data['text'].str.lower()

Посмотрим на соотношение классов:

In [7]:
comments_data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Наблюдается дисбаланс классов: негативные комментарии составляют только около 10.2% (16225) от всех (159571).

Напишем функцию для очистки текста от лишних символов:

In [8]:
def clean(text):
    
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    
    return text

Cохраним результат выполнения функции в новом столбце "text_clean", получим первые пять строк итоговой таблицы:

In [9]:
comments_data['text_clean'] = comments_data['text'].apply(clean)
comments_data.head()

,text,toxic,text_clean
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,d'aww! he matches this background colour i'm s...,0,daww he matches this background colour im seem...
2,"hey man, i'm really not trying to edit war. it...",0,hey man im really not trying to edit war its j...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i cant make any real suggestions on impro...
4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...


Напишем функцию для лемматизации английских текстов и исключения из него стоп-слов:

In [10]:
nlp = en_core_web_sm.load()

In [11]:
def lemmatize(text):
    lemma = []
    for token in nlp(text):
        if token.is_stop == False:
            lemma.append(token.lemma_)
    return " ".join(lemma)

Результат выполнения функции сохраним в столбце "text_lemma", получим первые 5 строк итоговой таблицы:

In [12]:
comments_data['text_lemma'] = comments_data['text_clean'].apply(lemmatize)
comments_data.head()

,text,toxic,text_clean,text_lemma
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation edit username hardcore metallica f...
1,d'aww! he matches this background colour i'm s...,0,daww he matches this background colour im seem...,daww match background colour be seemingly stic...
2,"hey man, i'm really not trying to edit war. it...",0,hey man im really not trying to edit war its j...,hey man be try edit war guy constantly remove ...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i cant make any real suggestions on impro...,not real suggestion improvement wonder secti...
4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...,sir hero chance remember page s


Переведем данные, необходимые для предсказания в нужный нам формат и сохраним в переменной corpus:

In [13]:
corpus = comments_data['text_lemma'].apply(lambda x: np.str_(x))

Разделим данные на обучающую и тестовую выборки в соотношении 80% к 20%:

In [14]:
corpus_train, corpus_test = train_test_split(corpus, test_size=0.2, random_state=12345)

In [15]:
target_train, target_test = train_test_split(comments_data['toxic'], test_size=0.2, random_state=12345)

Создадим счётчик:

In [16]:
count_tf_idf = TfidfVectorizer()

Посчитаем TF-IDF для корпуса текстов:

In [17]:
tf_idf_train = count_tf_idf.fit_transform(corpus_train)

In [18]:
tf_idf_test = count_tf_idf.transform(corpus_test)

# 2. Обучение

Рассмотрим три модели: градиентного бустинга, логистической регрессии и линейный метод опорных векторов. В связи с дисбалансом классов для каждой модели будем использовать параметр "class_weight='balanced'". Подбор гиперпараметров будем осуществлять с помощью GridSearchCV.

### 2.1 LGBMClassifier

Обучим модель с наилучшими параметрами посредством кросс-валидации:

In [19]:
model = LGBMClassifier(n_estimators = 150, class_weight='balanced')

In [20]:
tree_params = {'max_depth': [-1, 3, 5, 7],
               'learning_rate': [0.1, 0.3]}

In [21]:
LGBMC_grid = GridSearchCV(model, tree_params, cv=3, scoring='f1', verbose=False)

In [22]:
%%time
LGBMC_grid.fit(tf_idf_train, target_train)

CPU times: user 41min 23s, sys: 6.72 s, total: 41min 30s
Wall time: 41min 55s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LGBMClassifier(boosting_type='gbdt',
                                      class_weight='balanced',
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=150,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate'

Выведем на экран наилучшее сочетание параметров:

In [23]:
LGBMC_grid.best_params_

{'learning_rate': 0.3, 'max_depth': 7}

Выведем на экран лучшее значение F1-меры на валидационной выборке:

In [24]:
print("F1:", LGBMC_grid.best_score_)

F1: 0.7493180551129655


### 2.2 LogisticRegression

Обучим модель с наилучшими параметрами посредством кросс-валидации:

In [25]:
model_2 = LogisticRegression(class_weight='balanced')

In [26]:
tree_params = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
               'C': [0.1, 1, 10, 100]}

In [27]:
LG_grid = GridSearchCV(model_2, tree_params, cv=3, scoring='f1', verbose=False)

In [28]:
%%time
LG_grid.fit(tf_idf_train, target_train)

CPU times: user 14min 52s, sys: 9min 45s, total: 24min 38s
Wall time: 24min 40s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=False)

Выведем на экран параметр с наилучшим результатом:

In [29]:
LG_grid.best_params_

{'C': 10, 'solver': 'lbfgs'}

Выведем на экран лучшее значение F1-меры на валидационной выборке:

In [30]:
print("F1:", LG_grid.best_score_)

F1: 0.7675949491024238


### 2.3 LinearSVC

Обучим модель с наилучшими параметрами посредством кросс-валидации:

In [31]:
model_3 = LinearSVC(class_weight='balanced')

In [32]:
tree_params = {'C': [0.1, 1, 10, 100],
               'loss': ['hinge', 'squared_hinge']}

In [33]:
LSVC_grid = GridSearchCV(model_3, tree_params, cv=3, scoring='f1', verbose=False)

In [34]:
%%time
LSVC_grid.fit(tf_idf_train, target_train)

CPU times: user 1min 42s, sys: 0 ns, total: 1min 42s
Wall time: 1min 43s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LinearSVC(C=1.0, class_weight='balanced', dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100],
                         'loss': ['hinge', 'squared_hinge']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=False)

Выведем на экран наилучшее сочетание параметров:

In [35]:
LSVC_grid.best_params_

{'C': 1, 'loss': 'squared_hinge'}

Выведем на экран лучшее значение F1-меры на валидационной выборке:

In [36]:
print("F1:", LSVC_grid.best_score_)

F1: 0.7619703678532219


# 3. Тестирование

Проверим значение F1-меры модели логистической регрессии на тестовой выборке:

In [38]:
predicted_test = LG_grid.predict(tf_idf_test)
print("F1:", f1_score(target_test, predicted_test))

F1: 0.7615770609318996


# Вывод:

В процессе выполнения работы была выполнена предобработка данных: текст очищен от лишних символов, выполнена лемматизация и удаление стоп-слов английского текста при помощи библиотеки spacу. Была выявлена проблема дисбаланса классов и решена бутем применения параметра class_weight='balanced'. Для решения задачи классификации были обучены три модели: градиентного бустинга LGBMClassifier с F1-мерой равной 0.749, логистической регрессии LogisticRegression - 0.768 и линейный метод опорных векторов LinearSVC - 0.762. В качестве наилучшей была выбрана модель логистической регрессии с параметрами C = 10, solver = lbfgs, также LogisticRegression показала такой же хороший результат и на тестовой выборке: 0.762.